In [21]:
import tensorflow as tf
import os
import tf2onnx
import numpy as np

In [22]:
model = tf.keras.applications.resnet.ResNet50()
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

# 1. Save the model as keras model

In [4]:
model_dir = 'models/keras'
os.makedirs(model_dir, exist_ok=True)
input_model_path = os.path.join(model_dir, 'model.hdf5')
cmd_output_path = os.path.join(model_dir, 'cmd.onnx')
python_output_path = os.path.join(model_dir, 'py.onnx')
tf.keras.models.save_model(model, input_model_path)

## (1) Command Line way

In [ ]:
! python -m tf2onnx.convert --keras $input_model_path --output $cmd_output_path

## (2) Python way

In [ ]:
# convert with tfonnx function
signature = tf.TensorSpec([None, 224, 224, 3], tf.float32, name='input')
tf2onnx.convert.from_keras(model=model, input_signature=signature, output_path=python_output_path)

# 2. Saved the model as saved_model

In [8]:
model_dir = 'models/saved_model'
os.makedirs(model_dir, exist_ok=True)
input_model_path = os.path.join(model_dir, 'saved-model')
cmd_output_path = os.path.join(model_dir, 'cmd.onnx')
tf.saved_model.save(model, input_model_path)

INFO:tensorflow:Assets written to: models/saved_model/saved-model/assets


## (1) Command Line way

In [10]:
! python -m tf2onnx.convert --saved-model $input_model_path --output $cmd_output_path

/home/lmy/anaconda3/envs/tensorflow/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-17 17:21:25,964 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-10-17 17:21:29,459 - INFO - Signatures found in model: [serving_default].
2021-10-17 17:21:29,459 - WARNING - '--signature_def' not specified, using first signature: serving_default
2021-10-17 17:21:29,459 - INFO - Output names: ['predictions']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-17 17:21:31,189 - WARNING - From /home/lmy/anaconda3/envs/tensorflow/lib/python3.9/site-packages/tf2onnx/tf_loader.py:557: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.e

In [ ]:
signature = tf.TensorSpec([None, 224, 224, 3], tf.float32, name='input')
tf2onnx.convert.from_keras(model=model, input_signature=signature, output_path=python_output_path)

# 3. Inference all converted onnx models

In [39]:
# collect all onnx models
from glob import glob
import numpy as np
import onnxruntime
import onnx

np.random.seed(42)
image = np.random.rand(1, 224, 224, 3).astype(np.float32)

In [42]:
model_paths = glob('./models/**/*.onnx', recursive=True)
for model_path in model_paths:
    onnx_model = onnx.load(model_path)
    sess = onnxruntime.InferenceSession(model_path)
    result = sess.run(output_names=[onnx_model.graph.output[0].name], input_feed={onnx_model.graph.input[0].name: image})
    result = np.argmax(result)
    print('ONNX model: {}, result = {}'.format(model_path, result))

ONNX model: ./models/keras/py.onnx, result = 111
ONNX model: ./models/keras/cmd.onnx, result = 111
ONNX model: ./models/saved_model/cmd.onnx, result = 111
